In [1]:
import numpy as np
import pandas as pd
import pickle
import collections

In [2]:
from nltk.corpus import stopwords
import re
import string

In [3]:
sw = stopwords.words('portuguese')
def limpar_string(frase):
    frase = str(frase)
    frase = frase.lower()
    frase = re.sub(r'-', ' ', frase)
    frase = re.sub(r'<\w+>|\(\w+\)|[^\w\s]|\d|¹|²|³|º|ª|“|”', '', frase)
    frase = re.sub(r'\s+', ' ', frase)
    frase = frase.translate(str.maketrans('', '', string.punctuation))
    return ' '.join([word for word in frase.split() if word not in sw and len(word) > 2])

In [4]:
g1_df = pickle.load(open('data/g1_limpo.pkl', 'rb'))
boato_df = pickle.load(open('data/boato_limpo.pkl', 'rb'))

In [5]:
all_df = pd.concat([g1_df, boato_df])

In [6]:
all_df.reset_index(drop=True,  inplace=True)
all_df.reset_index(drop=False, inplace=True)

In [7]:
collections.Counter(all_df['fake'])

Counter({0: 4247, 1: 744})

In [8]:
all_df.head()

,index,date,text,title,len,fake
0,0,2019-01-06,presidente jair bolsonaro afirmou neste sábado...,Bolsonaro diz que quer manter estados e municí...,2724,0
1,1,2019-01-06,relator proposta reforma previdência comissão ...,Relator diz que déficit previdenciário de esta...,3706,0
2,2,2019-05-31,procuradoria geral república encaminhou manife...,PGR dá parecer a favor de indulto para ex-diri...,1166,0
3,3,2019-05-30,votação medida provisória estabelece pente fin...,Senado adia para a próxima segunda-feira votaç...,5166,0
4,4,2019-01-06,presidente jair bolsonaro afirmou entrevista c...,"A jornal argentino, Bolsonaro diz que Comissão...",1725,0


In [9]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from nltk.tokenize import RegexpTokenizer
from sklearn.pipeline import Pipeline
from nltk.stem import RSLPStemmer
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn import utils
import unicodedata

from sklearn.linear_model import SGDClassifier

In [10]:
rus = RandomUnderSampler()

X_all, y_all = rus.fit_resample(all_df[['index', 'text', 'title']], list(all_df['fake']))

print(X_all.shape)
print(collections.Counter(y_all))

(1488, 3)
Counter({0: 744, 1: 744})


In [11]:
vect = TfidfVectorizer(ngram_range=(1, 1), max_df=0.7, min_df=5, strip_accents='unicode')

clf = RandomForestClassifier(n_estimators=5000)

pipe = Pipeline([('vect', vect), ('clf', clf)])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(np.concatenate(([limpar_string(item) for i, item, title in X_all], [limpar_string(title) for i, item, title in X_all]), axis=0), np.concatenate((y_all, y_all), axis=0), test_size=0.3, random_state=42)

In [13]:
cv = cross_val_score(pipe, np.concatenate(([limpar_string(item) for i, item, title in X_all], [limpar_string(title) for i, item, title in X_all]), axis=0), np.concatenate((y_all, y_all), axis=0), n_jobs=-1, verbose=4, cv=10)

cv

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:  5.5min remaining:  3.7min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.8min finished


array([0.96308725, 0.98657718, 0.96979866, 0.9295302 , 0.92281879,
       0.8557047 , 0.90604027, 0.84228188, 0.82432432, 0.77702703])

In [14]:
cv.mean()

0.8977190277525848

In [15]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.7, max_features=None, min_df=5,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [16]:
pred = pipe.predict(X_test)

In [17]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.92      0.86      0.89       446
           1       0.87      0.92      0.90       447

   micro avg       0.89      0.89      0.89       893
   macro avg       0.89      0.89      0.89       893
weighted avg       0.89      0.89      0.89       893



In [18]:
print(confusion_matrix(y_test, pred))

[[385  61]
 [ 35 412]]


In [19]:
print(accuracy_score(y_test, pred))

0.8924972004479284


In [20]:
### Outros testes

In [21]:
### testando com o texto do dataframe inteiro

In [22]:
pred_all = pipe.predict(all_df['text'].apply(limpar_string))

In [23]:
print(classification_report(all_df['fake'], pred_all))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4247
           1       0.96      0.98      0.97       744

   micro avg       0.99      0.99      0.99      4991
   macro avg       0.98      0.99      0.98      4991
weighted avg       0.99      0.99      0.99      4991



In [24]:
print(confusion_matrix(all_df['fake'], pred_all))

[[4214   33]
 [  14  730]]


In [25]:
### testando com os titulos (manchetes) das notícias

In [26]:
pred_all = pipe.predict(all_df['title'].apply(limpar_string))

In [27]:
print(classification_report(all_df['fake'], pred_all))

              precision    recall  f1-score   support

           0       0.99      0.79      0.88      4247
           1       0.44      0.97      0.61       744

   micro avg       0.81      0.81      0.81      4991
   macro avg       0.72      0.88      0.74      4991
weighted avg       0.91      0.81      0.84      4991



In [28]:
print(confusion_matrix(all_df['fake'], pred_all))

[[3334  913]
 [  21  723]]


In [29]:
### testando com os textos dos conjuntos de treino + teste

In [42]:
pred_tt = pipe.predict([limpar_string(item[1]) for item in X_all])

In [43]:
print(classification_report(y_all, pred_tt))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       744
           1       1.00      0.98      0.99       744

   micro avg       0.99      0.99      0.99      1488
   macro avg       0.99      0.99      0.99      1488
weighted avg       0.99      0.99      0.99      1488



In [44]:
print(confusion_matrix(y_all, pred_tt))

[[742   2]
 [ 14 730]]


In [ ]:
### testando com algumas frases pontuais

In [45]:
def predict(text):
    text = limpar_string(text)
    return [pipe.predict([text]), pipe.predict_proba([text])]

In [46]:
# real:      verdadeiro
# predicted: verdadeiro
# fonte: https://g1.globo.com/politica/noticia/2019/06/01/a-jornal-argentino-bolsonaro-diz-que-comissao-da-verdade-e-mentirosa.ghtml

texto1 = """

O presidente Jair Bolsonaro (PSL) afirmou, em entrevista concedida ao jornal argentino ‘La Nacion’, publicada neste sábado (1º), que a Comissão da Verdade instituída para investigar violações aos direitos humanos durante a ditadura é “mentirosa”.

Questionado sobre a questão da Venezuela, Bolsonaro afirmou que o presidente Nicolás Maduro se mantém no poder porque cooptou o Exército, o que não ocorreu no Brasil.

“Isso, no Brasil, a esquerda não conseguiu fazer, apesar de todas as medidas tomadas contra nós, nem sequer com a mentirosa Comissão da Verdade, que contou histórias totalmente diferentes das que ocorreram no período de 1964 a 1985”, afirmou.

Bolsonaro disse ainda que a história do Brasil tem que começar a ser escrita com a verdade.

“Se o Partido dos Trabalhadores quisesse a verdade, o primeiro feito que deveria ser esclarecido seria o sequestro, tortura e execução, em 2002, do petista Celso Daniel, que estava para divulgar informação que acabaria com a campanha de Lula. E não se vê nenhum petista defendendo essa investigação, porque todos os indícios apontam que eles o mataram, entre tantos outros”, disse.

O jornal afirma que Bolsonaro, cinco meses depois de assumir o poder, está na “defensiva”, com a popularidade erodida, a economia em retrocesso e enfrenta obstáculos para impulsionar seus projetos no Congresso.

A entrevista foi concedida no Palácio do Planalto. O presidente disse também que deve visitar a Argentina na próxima semana para discutir um acordo entre o Mercosul e a União Europeia.

“A Argentina e o Brasil não podem voltar à corrupção do passado, uma corrupção desenfreada por busca de poder. Contamos com o povo argentino para eleger bem o presidente em outubro”, disse.

"""

predict(texto1)

[array([0]), array([[0.5466, 0.4534]])]

In [47]:
# real:      verdadeiro
# predicted: verdadeiro
# fonte: https://g1.globo.com/politica/noticia/2019/06/01/bolsonaro-diz-que-quer-manter-estados-e-municipios-na-reforma-da-previdencia.ghtml

texto2 = """

O presidente Jair Bolsonaro afirmou neste sábado (1º) que gostaria que o Congresso mantivesse estados e municípios dentro da reforma da Previdência enviada pelo governo. Ele ponderou, porém, que o "impasse" sobre o tema deve ser resolvido pelos parlamentares e, portanto, disse não ter "nada a ver com isso".

Nos últimos dias, deputados têm feito pressão para que o relator da reforma na comissão especial da Câmara, Samuel Moreira (PSDB-SP), deixe as previdências estaduais e municipais de fora do texto.

Na semana passada, o presidente da Câmara, Rodrigo Maia (DEM-RJ), admitiu que a pressão para retirar estados e municípios da reforma da Previdência é grande. Na avaliação de Maia, um dos principais fiadores da PEC que altera as regras previdenciárias, é "difícil" manter servidores estaduais na reforma.

"Isso está sendo acertado pela Câmara. O que nós gostaríamos é que fosse tudo junto, mas como tem partidos que querem que aprove [sem estados e municípios], e eles votando ao contrário, então há esse impasse dentro da Câmara, Eu não tenho nada a ver com isso. Não tenho nada a ver com isso, a Câmara que decide agora", disse Bolsonaro, após almoçar na casa de um amigo militar.

Questionado sobre se o governo teria alguma preferência, o presidente repetiu o que vem dizendo em declarações públicas, de que gostaria de ver aprovado o texto original da proposta.


"Eu quero aprovar a reforma basicamente como chegou lá, eu espero que o pessoal se entenda. Tem parlamentar reclamando: 'Olha, a gente quer votar, mas o colega de tal partido quer que ela passe, mas ele votou no contrário porque tem algum desgaste no estado'. É esse o problema que está acontecendo dentro da Câmara", afirmou.

O presidente disse ainda que todos os demais projetos econômicos "nascerão" da reforma da Previdência. Para ele, a PEC é a "reforma mãe" do governo.

Relator
Também neste sábado, o relator da proposta reuniu-se em Brasília com integrantes da área técnica do governo. Após o encontro, Samuel Moreira afirmou que o déficit previdenciário de estados e prefeituras soma R$ 96 bilhões por ano.

Ele disse, porém, que isso é um assunto "polêmico e grave" e que ainda não decidiu se irá mexer neste ponto no parecer final.

Indagado pelos repórteres ao final da reunião com a área técnica do governo federal sobre a pressão política para retirar estados e municípios da reforma, ele disse que "não há qualquer cálculo eleitoral que possa ser maior que a responsabilidade nesse momento".

A expectativa do relator é de apresentar o parecer final à comissão especial no fim da próxima semana. Até lá, ele pretende continuar conversando e negociando com líderes partidários para tentar construir um texto que atraia os votos do Centrão.

"""

predict(texto2)

[array([0]), array([[0.8212, 0.1788]])]

In [48]:
# real:      falso
# predicted: falso
# fonte: https://www.boatos.org/politica/editorial-o-globo-pede-intervencao.html

texto3 = """

EDITORIAL DE O GLOBO Segunda-feira 20.05.2019. Não vivemos mais um regime Presidencialista! Que eu saiba, questões de gestão inerentes ao exercício do Poder determinadas pelo Presidente da República, jamais foram contestadas ao longo do tempo. Nos estranhos dias atuais, onde a esquerda não se conforma em perder o Poder e acha que democracia é o poder que emana do lado esquerdo do povo, o Presidente da República, Jair Bolsonaro, legitimamente eleito por mais de 57 milhões de brasileiros, virou um títere, um boneco de ventríloquo nas mãos de Congressistas, Juízes, Procuradores e Ministros de má fé suportados pela mídia. […]

É, meus amigos, acho que este País só tem um jeito: Intervenção militar e fechamento de Congresso, Judiciário e o escambau. Ai esses filhos da puta desses políticos corruptos, esses juizinhos engajados de merda, esses artistas e essa mídia de bosta, toda essa gentalha que não se conforma em perder as eleições, que não admite que o Brasil abomina a esquerda e que não quer ver um governo democrático funcionar, vai ver o que é bom e ai, sim, vai poder discutir, na prática e com conhecimento de causa, aqui e agora, se o que acontecer é golpe, é revolução, é ditadura, é contra-revolução ou é simplesmente a reação de um povo de saco cheio, como o foi em 64. ps: se concordar, compartilhe, pois estou bloqueado e este perfil é super limitado, não atinge nem 100 pessoas. Obrigado.

"""

predict(texto3)

[array([1]), array([[0.2692, 0.7308]])]

In [49]:
# real:      falso
# predicted: falso
#fonte: rodney

texto4 = """

lula e bolsonaro são avistados na fronteira do mexico

"""
predict(texto4)

[array([1]), array([[0.093, 0.907]])]

In [50]:
# real:      falso
# predicted: falso
# fonte: rodney

texto5 = """

Eles irão tirar a frase deus seja louvado e colocarão brasil pais lgbt claramente numa tentativa de destruir as familia de bem que ajudaram a construir esse pais e manter a familia que deus criou

"""
predict(texto5)

[array([1]), array([[0.2348, 0.7652]])]

In [51]:
texto6 = """

Não é fake não, direitas evangélicas! O que Bolsonaro queria, procurando fotos do Pablo Vitar com a b… grande e gostosa? Twitter na real kkkk 

"""

predict(texto6)

[array([1]), array([[0.0604, 0.9396]])]

In [52]:
texto7 = """

A presidente nacional do PT, Gleisi Hoffmann, negou neste domingo, 2, que haja uma separação entre a pauta do "Lula Livre" e as demais bandeiras da esquerda, como a da defesa da educação. "Lula e educação são pautas inseparáveis. Essa moçada está indo às ruas pelo legado que ele deixou neste País", disse a petista, uma das únicas autoridades do partido a estar presente no festival Lula Livre, que não contou com discursos de políticos no palco. VEJA TAMBÉM Festival Lula Livre não destaca outras bandeiras de esquerda "Para nós do PT, 'Lula Livre' é estratégico, nós achamos que não há salvação da democracia com Lula preso. Nós sempre levaremos a pauta do 'Lula Livre' junto com a pauta da defesa da educação", disse Hoffmann. Perguntada se os partidos aliados também fazem a mesma associação, a ex-senadora respondeu que "muitos têm levado (uma agenda em conjunto coma outra)". Citou o PCdoB e o PSOL e afirmou que Carlos Lupi, presidente do PDT, visitou Lula na prisão. Além de não contar com discursos políticos, o ato deste domingo não enfatizou nenhuma outra grande pauta da esquerda, como a defesa das instituições públicas de ensino e a oposição à reforma da Previdência. O Broadcast/Estadão apurou que a organização de atos paralelos é uma estratégia da oposição para manter agenda de protestos pró-Lula sem que essa pauta isole quem não simpatiza com o ex-presidente da agenda anti-Bolsonaro. Para Guilherme Boulos, que foi candidato do PSOL à Presidência da República ano passado, as manifestações dos dias 15 e 30 de maio - contra o contingenciamento de recursos de instituições de ensino anunciado pelo governo Jair Bolsonaro - foram "com centro na luta pela educação". "Agora aqui é um evento pensado com o tema do 'Lula Livre'". Segundo Paulo Okamotto, presidente dos Instituto Lula, o evento deste domingo não conta com discursos por se tratar de um "ato lúdico". Ele disse ainda que os políticos com mandato ajudaram a fazer o contato com os artistas e a chamar público pelas redes sociais.

"""

predict(texto7)

[array([0]), array([[0.566, 0.434]])]

In [53]:
texto8 = """

Bolsonaro leva facada em atentado durante campanha em Juiz de Fora

"""
predict(texto8)

[array([1]), array([[0.0866, 0.9134]])]

In [54]:
texto9 = """

O presidente Jair Bolsonaro comentou a recente notícia de que a o PIB brasileiro sofreu uma queda de 0,2% no primeiro trimestre do ano, reiterando a sua confiança no ministro Paulo Guedes.

Ao ser perguntado por jornalistas sobre as projeções para a economia brasileira, Bolsonaro relacionou as projeções econômicas brasileiras com fatores externos e reiterou a necessidade de aprovara reforma da Previdência.

"Já falei que não entendia de economia? Quem entendia afundou o Brasil, eu confio 100% na economia do Paulo Guedes [...] A gente quer melhorar os nossos índices aqui, agora passa por questões até externas", disse Bolsonaro, durante um almoço na casa de um amigo, em Brasília.

De acordo com ele, está "descartada qualquer possibilidade de novo imposto ou majorar qualquer imposto, isso não existe".

Além disso, ele destacou que o governo está trabalhando para garantir a promessa de pagar a 13º parcela do Bolsa Família por meio do combate a fraudes no benefício.

Um estudo do IBGE divulgada em 29 de maio mostrou que o PIB brasileiro sofre uma queda de 0,2% no primeiro trimestre de 2019 em comparação com o último trimestre do ano passado.

Trata-se da primeira queda do PIB desde o quarto trimestre de 2016. No último trimestre de 2018 foi registrado o aumento do PIB de 0,1% em comparação com o terceiro trimestre de 2018.


"""

predict(texto9)

[array([1]), array([[0.3994, 0.6006]])]

In [55]:
texto10 = """

Lula tem direito a progredir para o regime semiaberto, diz MPF em parecer ao STJ

"""

predict(texto10)

[array([1]), array([[0.3634, 0.6366]])]

In [56]:
elpais_df = pd.read_json('data/elpais.json')

In [57]:
len(elpais_df)

11681

In [58]:
elpais_df.head()

,date,text,title
0,\n4 JUN 2019 - 14:27\t\t\t\t\t,[O atual mandato presidencial no Brasil começo...,Começam a soar os alarmes sobre a sustentabili...
1,\n2 JUN 2019 - 12:49\t\t\t\t\t,"[Poucas horas antes de milhares de , manifesta...","Corte ou contingenciamento, quem está certo na..."
2,\n3 JUN 2019 - 11:00\t\t\t\t\t,[Se o tamanho de uma figura pública se mede pe...,Trump insulta prefeito de Londres no início de...
3,\n26 MAI 2019 - 16:18\t\t\t\t\t,[Um dos tantos fenômenos imparáveis trazidos p...,O líder e eu (e ninguém no meio)
4,\n3 JUN 2019 - 16:22\t\t\t\t\t,"[Após semanas de audiências públicas, o projet...",A reforma da Previdência pesará mais sobre os ...


In [59]:
elpais_df['text'] = [''.join(sent) for sent in elpais_df['text']]

In [60]:
elpais_df.head()

,date,text,title
0,\n4 JUN 2019 - 14:27\t\t\t\t\t,O atual mandato presidencial no Brasil começou...,Começam a soar os alarmes sobre a sustentabili...
1,\n2 JUN 2019 - 12:49\t\t\t\t\t,Poucas horas antes de milhares de manifestante...,"Corte ou contingenciamento, quem está certo na..."
2,\n3 JUN 2019 - 11:00\t\t\t\t\t,Se o tamanho de uma figura pública se mede pel...,Trump insulta prefeito de Londres no início de...
3,\n26 MAI 2019 - 16:18\t\t\t\t\t,Um dos tantos fenômenos imparáveis trazidos pe...,O líder e eu (e ninguém no meio)
4,\n3 JUN 2019 - 16:22\t\t\t\t\t,"Após semanas de audiências públicas, o projeto...",A reforma da Previdência pesará mais sobre os ...


In [62]:
elpais_df['text'] = elpais_df['text'].apply(limpar_string)

In [63]:
elpais_df.head()

,date,text,title
0,\n4 JUN 2019 - 14:27\t\t\t\t\t,atual mandato presidencial brasil começou pouc...,Começam a soar os alarmes sobre a sustentabili...
1,\n2 JUN 2019 - 12:49\t\t\t\t\t,poucas horas antes milhares manifestantes irem...,"Corte ou contingenciamento, quem está certo na..."
2,\n3 JUN 2019 - 11:00\t\t\t\t\t,tamanho figura pública mede importância inimig...,Trump insulta prefeito de Londres no início de...
3,\n26 MAI 2019 - 16:18\t\t\t\t\t,tantos fenômenos imparáveis trazidos revolução...,O líder e eu (e ninguém no meio)
4,\n3 JUN 2019 - 16:22\t\t\t\t\t,após semanas audiências públicas projeto refor...,A reforma da Previdência pesará mais sobre os ...


In [64]:
elpais_pred = pipe.predict(elpais_df['text'])

In [65]:
elpais_df['fake'] = 0

In [66]:
print(classification_report(elpais_df['fake'], elpais_pred))

              precision    recall  f1-score   support

           0       1.00      0.76      0.86     11681
           1       0.00      0.00      0.00         0

   micro avg       0.76      0.76      0.76     11681
   macro avg       0.50      0.38      0.43     11681
weighted avg       1.00      0.76      0.86     11681



c:\users\hidea\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [67]:
print(confusion_matrix(elpais_df['fake'], elpais_pred))

[[8874 2807]
 [   0    0]]
